###  Moive Recommendation

#### Dataset chocie
- Dataset: [MovieLens dataset](https://grouplens.org/datasets/movielens/latest/)
- For the Part 1, we use the **complete dataset** (27753444 rows of ratings)
- For the Part 2, in order to save computation time, we use the **small dataset** (100836 rows of ratings, about 0.36% of the complete dataset)

In [2]:
# import packages 

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import os
os.environ["PYSPARK_PYTHON"] = "python3"

from pyspark.sql import SparkSession
import pyspark.sql.functions as fn
from pyspark.sql.types import *

## Part 1: Data ETL and Data Exploration

In [4]:
# setup spark session
spark = SparkSession \
    .builder \
    .appName("moive analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [5]:
# filepath and load dataset.
filepath_small = "/FileStore/tables/ml-latest-small/"  # small dataset
filepath_large = "/FileStore/tables/ml-latest/"  # large dataset

filepath = filepath_large # we use large dataset for Part 1, data exploration analysis
# filepath = filepath_small # we use small dataset for Part 2, ALS and rating prediction

movies = spark.read.load(filepath + "movies.csv", format='csv', header = True)
ratings = spark.read.load(filepath + "ratings.csv", format='csv', header = True)
links = spark.read.load(filepath + "links.csv", format='csv', header = True)
tags = spark.read.load(filepath + "tags.csv", format='csv', header = True)

In [6]:
movies.printSchema()
ratings.printSchema()
links.printSchema()
tags.printSchema()

root
-- movieId: string (nullable = true)
-- title: string (nullable = true)
-- genres: string (nullable = true)

root
-- userId: string (nullable = true)
-- movieId: string (nullable = true)
-- rating: string (nullable = true)
-- timestamp: string (nullable = true)

root
-- movieId: string (nullable = true)
-- imdbId: string (nullable = true)
-- tmdbId: string (nullable = true)

root
-- userId: string (nullable = true)
-- movieId: string (nullable = true)
-- tag: string (nullable = true)
-- timestamp: string (nullable = true)

In [7]:
print ("There are %d movies" % movies.count())
print ("There are %d rows of ratings" % ratings.count())

There are 58098 movies
There are 27753444 rows of ratings

In [8]:
# calculating max of a column: https://stackoverflow.com/questions/33224740/best-way-to-get-the-max-value-in-a-spark-dataframe-column

print('For the users that rated movies and the movies that were rated:')
print('Minimum number of ratings per user is {}'\
      .format(
        ratings.groupBy(ratings.userId).count().agg({'count': 'min'}).collect()[0][0]
      ))
print('Minimum number of ratings per movie is {}'\
      .format(
        ratings.groupBy(ratings.movieId).count().agg({'count': 'min'}).collect()[0][0]
      ))

For the users that rated movies and the movies that were rated:

In [9]:
print('{} out of {} movies are rated by only one user'\
      .format(
            ratings.groupBy("movieId").count().filter("count == 1").count(), 
            ratings.select('movieId').distinct().count()
          )
       )

10155 out of 53889 movies are rated by only one user

## Part 1: Spark SQL and OLAP

### Q1: The number of Users

In [12]:
print ("number of distinct users", ratings.select('userId').distinct().count())

number of distinct users 283228

### Q2: The number of Movies

In [14]:
print ("number of distinct movies", movies.select('movieId').distinct().count())

number of distinct movies 58098

### Q3:  How many movies are rated by users? List movies not rated before

In [16]:
print ("number of rated movies", ratings.select('movieId').distinct().count())

number of rated movies 53889

In [17]:
df_q3 = movies.join(
    ratings.select('movieId').distinct(), 
    on = "movieId",
    how = "left_anti"
  )
# left_anti join: https://stackoverflow.com/a/39889263/7065092

display(
  df_q3
)


movieId,title,genres
135352,Heroes of the West (1965),(no genres listed)
135901,Count the Hours (1953),Crime
138076,Diary of a Tired Black Man (2009),Comedy|Drama|Romance
145275,Live Wires (1946),Action|Thriller
151859,Hart to Hart Returns (1993),Adventure|Crime|Mystery
156821,Yakeen (1969),Action|Crime|Mystery|Romance|Thriller
159526,Black Cobra Woman (1976),Drama|Horror
163288,Those Three French Girls (1930),(no genres listed)
165977,Behind the Make-Up (1930),Drama
174251,The Cure (1924),Animation|Comedy


In [18]:
print ("number of movies not rated before:", df_q3.count())
print ("This is %0.2lf%% of all movies" % (df_q3.count() / movies.select('movieId').distinct().count() * 100))

number of movies not rated before: 4209
This is 7.24% of all movies

### Q4: List Movie Genres

In [20]:
# have a look at the original string data
display(movies.select("genres").limit(10))

genres
Adventure|Animation|Children|Comedy|Fantasy
Adventure|Children|Fantasy
Comedy|Romance
Comedy|Drama|Romance
Comedy
Action|Crime|Thriller
Comedy|Romance
Adventure|Children
Action
Action|Adventure|Thriller


In [21]:
# First, split string with "|"
split_col = fn.split(movies.genres, "\\|") 

# Find out the largest number genres a movie has
split_length = movies.withColumn("split_size", fn.size(split_col)).agg({"split_size" : "max"}).collect()[0][0]
print ("a movie has at most {} genres in our dataset".format(split_length))
print ("on average, a movie has %0.2lf genres" % (movies.withColumn("split_size", fn.size(split_col)).agg({"split_size" : "mean"}).collect()[0][0]))

a movie has at most 10 genres in our dataset
on average, a movie has 1.83 genres

In [22]:
df_pd_temp = movies.withColumn("split_size", fn.size(split_col)).select("split_size").orderBy("split_size").toPandas()


In [23]:
fig, ax = plt.subplots()
plt.hist(df_pd_temp.split_size, normed = True)
plt.title("histogram for the number of genres one movie has")
plt.xlabel("number of genres of one movie")
plt.ylabel("percentage of movies")
display(fig)

### We can see that more than 50% of the movies has only 1 genre label.

In [25]:
# Find all the unique genres by combining the splitted genre strings

df_q4 = None

for i in range(split_length):
    df_temp = movies \
                .withColumn("split_genre", split_col.getItem(i)) \
                .select("split_genre") \
                .distinct()
    if df_q4 is None:
        df_q4 = df_temp
    else:
        df_q4 = df_q4.join(df_temp, 
                           on = "split_genre", 
                           how = "full")

# remove null and sort
df_q4 = df_q4 \
        .filter(df_q4.split_genre.isNotNull()) \
        .orderBy("split_genre") 

In [26]:
display(df_q4)

split_genre
(no genres listed)
Action
Adventure
Animation
Children
Comedy
Crime
Documentary
Drama
Fantasy


In [27]:
print ("There are {} unique genres".format(df_q4.count()))

There are 20 unique genres

In [28]:
movies_pd = movies.toPandas()
unique_genres = sorted(movies_pd.genres.str.split("|", expand = True).stack().unique())
print (unique_genres)
print ("There are {} genres in total".format(len(unique_genres)))

['(no genres listed)', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
There are 20 genres in total

### Q5: Movie for Each Category

In [30]:
display(movies.where("genres like '%Action%'"))

movieId,title,genres
6,Heat (1995),Action|Crime|Thriller
9,Sudden Death (1995),Action
10,GoldenEye (1995),Action|Adventure|Thriller
15,Cutthroat Island (1995),Action|Adventure|Romance
20,Money Train (1995),Action|Comedy|Crime|Drama|Thriller
23,Assassins (1995),Action|Crime|Thriller
42,Dead Presidents (1995),Action|Crime|Drama
44,Mortal Kombat (1995),Action|Adventure|Fantasy
51,Guardian Angel (1994),Action|Drama|Thriller
66,Lawnmower Man 2: Beyond Cyberspace (1996),Action|Sci-Fi|Thriller


In [31]:
# count the number of movies for each genre
unique_genres_count = [movies.where("genres like '%{}%'".format(genre)).count() for genre in unique_genres]

# save result to a spark dataframe
df_q5 = spark.createDataFrame(
            zip(unique_genres, unique_genres_count),
            schema = StructType(
                     [StructField("genre", StringType()),
                    StructField("movie_count", IntegerType())])
)

In [32]:
display(df_q5.orderBy("movie_count", ascending = False))

genre,movie_count
Drama,24144
Comedy,15956
Thriller,8216
Romance,7412
Action,7130
Horror,5555
Documentary,5118
Crime,5105
(no genres listed),4266
Adventure,4067


## Now let's find out which genres are more likely to be in the same movie
For convenience, we're using pandas now

In [34]:
# create binary features for each unique genre
genre_dict = {genre:movies_pd.genres.apply(lambda s: s.find(genre) >= 0) for genre in unique_genres if genre != "(no genres listed)"}
genre_df_pd = pd.DataFrame(data = genre_dict)

In [35]:
# plot correlation matrix of the genres
corr = genre_df_pd.corr()
fig, ax = plt.subplots(figsize = (12, 8))
plt.clf()
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, 
            cmap = cmap, 
            vmin = -0.25, vmax = 0.25, 
            center = 0)
plt.title("Correlation of the genres")
display(fig)

In [36]:
# cluster genres using hierarchical clustering, in seaborn.clustermap
corr = genre_df_pd.corr()
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.clustermap(corr, 
               cmap = cmap,
               vmin = -0.25, vmax = 0.25,
              center = 0
              )
# plt.savefig("movie_genre_clustermap.png", bbox_inches = "tight")

Out[ 74 ]: <seaborn.matrix.ClusterGrid at 0x7f52cdab3f28>

![1](https://gist.github.com/huizew/2284763b33775800c695af4ee170a6c2/raw/af87a7e76f817ca0fa0617e744a14c42df803d24/movie_genre_clustermap.png)

#### What we can see from genre correlation plot
- On the lower right corner, 
  - `Thriller`, `Mystery` and `Crime` are very closely related to each other
  - `Horror` frequently appear with `Thriller` and `Mystery`, but not `Crime`
  - `Film-Noir` are related with `Thriller`, `Mystery` and `Crime`, but not `Horror`
- In the middle region,
  - `IMAX`, `Sci-Fi`, `Action` and `Adventure` are tightly coupled with each other
  - `Fantasy`, `Animation` and `Children` are very closely related
  - `Adventure` is very strongly related with six other genres in this middle region
- In the upper left region,
  - `Musical`, `Comedy` and `Romance` are often labeled to the same movies
  - `Drama` and `War` goes hand in hand
  - `Drama` and `Documentary` has the strongest negative correlation, as expected

#### Conclusions for Part 1
- Most of the movies are rated, and only 7.24% are not rated
- Some of the rated movies are only rated by 1 user (10155 out of 53889 movies). 
- When building a prediction model, we can consider throwing away the movies that are only rated by 1 user (or fewer than a certain number of users), because these ratings are very subjective and has high variance, and very difficult to learn. Or we can treat popular movies and unpopular movies different.
- There are 20 unique genres in our dataset, including "no genres listed".
- On average, a movie has 1.83 genre labels, and more than 50% of the movies has only 1 genre label.
- Therefore, when doing matrix factorization, we can use these information to guide the choice of the number of latent factors.
- We also plotted the correlation matrix of the genres, to see which genres are more likely to be in the same movie. And we can see some interesting clusters.

## Part2: Spark ALS based approach for training model
- We use Spark ML ALS model (dataframe-based API), instead of the RDD-based API. [Reference 1](https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/4999972933037924/899848065201823/8135547933712821/latest.html), [Reference 2](https://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html)
- We first split the dataset into training set, validation set and test set, with proportions 60%, 20%, 20%.
- Then we do a parameter grid search (regularization parameter and rank), and use validation set performance to find the optimal set of hyperparameter.
- We also plot the learning curve and show how training set error converges as we increase iteration time

In [41]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [42]:
# we use the small dataset because of computational resource limitations
filepath = filepath_small
ratings = spark.read.load(filepath + "ratings.csv", format='csv', header = True)

In [43]:
ratings = ratings \
            .withColumn("userId", ratings.userId.cast(IntegerType())) \
            .withColumn("movieId", ratings.movieId.cast(IntegerType())) \
            .withColumn("rating", ratings.rating.cast(FloatType())) 

In [44]:
train, validation, test = ratings.randomSplit([6.0,2.0,2.0], seed = 7856)

train.cache()
validation.cache()
test.cache()

Out[ 75 ]: DataFrame[userId: int, movieId: int, rating: float, timestamp: string]

### ALS Model Selection and Evaluation

With the ALS model, we can use a grid search to find the optimal hyperparameters.

In [46]:
def train_ALS(train_data, validation_data, num_iters, reg_list, rank_list):
    min_error = float('inf')
    best_rank = -1
    best_regularization = 0
    best_model = None
    for rank in rank_list:
        for reg in reg_list:
            # write your approach to train ALS model
            # make prediction
            # get the rating result
            # get the RMSE
            als = ALS(maxIter=num_iters, regParam=reg, rank=rank,
                      userCol="userId", itemCol="movieId", ratingCol="rating",
                      coldStartStrategy="drop")
            model = als.fit(train_data)
            # Evaluate the model by computing the RMSE on the test data
            predictions = model.transform(validation_data)
            evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
            error = evaluator.evaluate(predictions)
            print ('{} latent factors and regularization = {}: validation RMSE is {}'.format(rank, reg, error))
            if error < min_error:
                min_error = error
                best_rank = rank
                best_regularization = reg
                best_model = model
    print ('\nThe best model has {} latent factors and regularization = {}'.format(best_rank, best_regularization))
    return best_model

In [47]:
num_iterations = 10
rank_list = [6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]
reg_list = [0.05, 0.1, 0.2, 0.4, 0.8]

import time
start_time = time.time()
final_model = train_ALS(train, validation, num_iterations, reg_list, rank_list)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

6 latent factors and regularization = 0.05: validation RMSE is 0.9728484470027742
6 latent factors and regularization = 0.1: validation RMSE is 0.9106082642382424
6 latent factors and regularization = 0.2: validation RMSE is 0.8901343818449075
6 latent factors and regularization = 0.4: validation RMSE is 0.9588577484767883
6 latent factors and regularization = 0.8: validation RMSE is 1.1803166052080318
8 latent factors and regularization = 0.05: validation RMSE is 0.985948719847176
8 latent factors and regularization = 0.1: validation RMSE is 0.912296900477279
8 latent factors and regularization = 0.2: validation RMSE is 0.8916161975625796
8 latent factors and regularization = 0.4: validation RMSE is 0.9598080583835218
8 latent factors and regularization = 0.8: validation RMSE is 1.1802933081819595
10 latent factors and regularization = 0.05: validation RMSE is 0.9876907854687087
10 latent factors and regularization = 0.1: validation RMSE is 0.9080839534551534
10 latent factors and regularization = 0.2: validation RMSE is 0.8900122827476168
10 latent factors and regularization = 0.4: validation RMSE is 0.9593363427971594
10 latent factors and regularization = 0.8: validation RMSE is 1.1802981767472829
12 latent factors and regularization = 0.05: validation RMSE is 1.0025589400098305
12 latent factors and regularization = 0.1: validation RMSE is 0.9101812701700668
12 latent factors and regularization = 0.2: validation RMSE is 0.8901818774198641
12 latent factors and regularization = 0.4: validation RMSE is 0.9594580375682211
12 latent factors and regularization = 0.8: validation RMSE is 1.1802997351973066
14 latent factors and regularization = 0.05: validation RMSE is 0.992799012981669
14 latent factors and regularization = 0.1: validation RMSE is 0.9050334743494243
14 latent factors and regularization = 0.2: validation RMSE is 0.8888561537857205
14 latent factors and regularization = 0.4: validation RMSE is 0.9591834061601024
14 latent factors and regularization = 0.8: validation RMSE is 1.1802993285459458
16 latent factors and regularization = 0.05: validation RMSE is 1.0030911259321418
16 latent factors and regularization = 0.1: validation RMSE is 0.9049385252271307
16 latent factors and regularization = 0.2: validation RMSE is 0.8904714201390124
16 latent factors and regularization = 0.4: validation RMSE is 0.9597231205632804
16 latent factors and regularization = 0.8: validation RMSE is 1.1803000623354836
18 latent factors and regularization = 0.05: validation RMSE is 1.0067388495552754
18 latent factors and regularization = 0.1: validation RMSE is 0.9074329092696501
18 latent factors and regularization = 0.2: validation RMSE is 0.8901150898780428
18 latent factors and regularization = 0.4: validation RMSE is 0.9593028819150996
18 latent factors and regularization = 0.8: validation RMSE is 1.180297190681933
20 latent factors and regularization = 0.05: validation RMSE is 1.0015661664851248
20 latent factors and regularization = 0.1: validation RMSE is 0.905851238347552
20 latent factors and regularization = 0.2: validation RMSE is 0.8908848089522832
20 latent factors and regularization = 0.4: validation RMSE is 0.9593341653423385
20 latent factors and regularization = 0.8: validation RMSE is 1.1802978888741271
22 latent factors and regularization = 0.05: validation RMSE is 1.0035812723515203
22 latent factors and regularization = 0.1: validation RMSE is 0.9072207387315855
22 latent factors and regularization = 0.2: validation RMSE is 0.8906272383374473
22 latent factors and regularization = 0.4: validation RMSE is 0.9592593079963677
22 latent factors and regularization = 0.8: validation RMSE is 1.1802990462416658
24 latent factors and regularization = 0.05: validation RMSE is 1.0013129658066684
24 latent factors and regularization = 0.1: validation RMSE is 0.9063316761504462
24 latent factors and regularization = 0.2: validation RMSE is 0.8909154532285316
24 latent factors and regularization = 0.4: validation RMSE is 0.9593

### Parameter grid search result
- The best rank (number of latent factors) is 14, and the best regularization parameter is 0.2
- It roughly agrees with the total number of movie genres (20) that we found in Part 1.

In [49]:
def plot_learning_curve(num_iters_list, train_data, reg, rank):
    error_list = []
    for num_iters in num_iters_list:
        als = ALS(maxIter=num_iters, regParam=reg, userCol="userId", itemCol="movieId", ratingCol="rating",
                coldStartStrategy="drop")
        model = als.fit(train_data)
        # Evaluate the model by computing the RMSE on the test data
        predictions = model.transform(train_data)
        evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
        error = evaluator.evaluate(predictions)
        print ('{} iterations, {} latent factors and regularization = {}: training set RMSE is {}'.format(num_iters, rank, reg, error))
        error_list.append(error)
    fig, ax = plt.subplots()
    plt.plot(num_iters_list, error_list, marker = "o")
    plt.xlabel("number of iterations")
    plt.ylabel("training set RMSE")
    plt.title("learning curve of ALS iterations")
    plt.ylim([0, 2.5])
    display(fig)
    return error_list

In [50]:
num_iters_list = [1, 2, 5, 8, 10]
error_list = plot_learning_curve(num_iters_list = num_iters_list, train_data = train,  
                    reg = 0.2, rank = 14)
# write your function to plot the learning curve 

### We can see that 10 iteration times should be enough

### Model testing on the test data

- After finding the best hyperparameter, we should re-train the model with training + validation dataset, which is 80% of all data
- This gives better performance than just using the "final_model", which is only trained on the training set (60% data)

In [53]:
predictions = final_model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
error = evaluator.evaluate(predictions)
print ("If we don't retrain the best model, and just use it on 60% data,  the test set RMSE is {}".format(error))

If we don't retrain the best model, and just use it on 60% data, the test set RMSE is 0.8892482518086302

In [54]:
als = ALS(maxIter=10, regParam=0.2, rank=14,
          userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(train.union(validation))
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
error = evaluator.evaluate(predictions)
print ("If we do it properly, and re-train the best model on training set + validation set, the test set RMSE is {}, which is an significant improvement".format(error))


If we do it properly, and re-train the best model on training set + validation set, the test set RMSE is 0.8759059092854435, which is an significant improvement

### conclusions for Part 2
- In Part 2, we played with three parameters of ALS (`rank`, `regParam` and `maxIt`)
- We did parameter grid search for `rank` and `regParam`, and tested on validation set. We could also do cross-validation, which will be slower computationally.
- We found that the best `rank` parameter is `14`, which roughly agrees with the number of unique genres of the movies. It provides a way to intuitively understand the matrix factorization. 
- We also plotted a learning curve against `maxIt` to show that 10 iterations should be enough.
- After finding the best set of hyperparameters, we re-trained the model on training set + validation set, and tested the performance on test set.
- It works! Cool!